In [7]:
import pyodbc
import json

server = 'KPI-MONITOR'
database = 'MEI2'
username = 'efqm'
password = 'mpeiR@dar'
filename = 'out_11_12_2024_АСУТП.json'

# Define the connection string and connect to the database
connection_string = f'DRIVER=SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'
connection = pyodbc.connect(connection_string)

# Load the JSON data from a file
with open(filename, 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

# Get the required subdivision from the JSON (it should be "АЭП" in this case)
subdivision_name = 'АСУТП'

In [13]:
import pyodbc
import json
import os

server = 'KPI-MONITOR'
database = 'MEI2'
username = 'efqm'
password = 'mpeiR@dar'

# Папка, содержащая файлы JSON
json_folder = 'JSON'

# Подключение к базе данных
connection_string = f'DRIVER=SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'
connection = pyodbc.connect(connection_string)

# Получаем список всех файлов в папке JSON
json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

# Запрашиваем у пользователя название файла для subdivision_name
print("Список доступных файлов:")
for idx, file in enumerate(json_files, 1):
    print(f"{idx}. {file}")

file_index = int(input("Введите номер файла, который хотите использовать: ")) - 1
filename = os.path.join(json_folder, json_files[file_index])

# Загружаем данные JSON из выбранного файла
with open(filename, 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

# Запрашиваем название подразделения для subdivision_name
subdivision_name = input("Введите название подразделения: ")

# Теперь у вас есть переменные filename (путь к файлу) и subdivision_name
print(f"Выбран файл: {filename}")
print(f"Подразделение: {subdivision_name}")

# Пример работы с JSON и базы данных, продолжение вашего кода...
# Вы можете добавить проверку данных, как в предыдущем примере
if subdivision_name not in json_data:
    print(f"Subdivision '{subdivision_name}' not found in the JSON file.")
else:
    # Retrieve pages from the JSON for this subdivision
    pages = json_data[subdivision_name].get('pages', [])

    # Query the database to get the necessary data
    cursor = connection.cursor()
    sub_query = f"""
        SELECT page, page_id, question_id, question_description, option_id, option_name, answer
        FROM [MEI2].[dbo].[EFQM_SurveyAnswers]
        WHERE [SubdivisionShortName] = '{subdivision_name}'
    """
    results = cursor.execute(sub_query).fetchall()

    # Prepare a dictionary to store the database results by page_id, question_id, and option_id
    db_data = {}
    for row in results:
        page = row.page
        page_id = row.page_id
        question_id = row.question_id
        option_id = row.option_id
        option_name = row.option_name

        # Add the row data to the dictionary
        if page_id not in db_data:
            db_data[page_id] = {}
        if question_id not in db_data[page_id]:
            db_data[page_id][question_id] = {}
        db_data[page_id][question_id][option_id] = option_name

    # Check if all the data in the JSON exists in the database query result
    missing_data = []

    for page in pages:
        page_id = page.get('page_id')
        page_name = page.get('page')
        
        # Check for questions in each page
        for question in page.get('questions', []):
            question_id = question.get('question_id')
            question_description = question.get('question_description')

            # Check for options in each question
            for option in question.get('options', []):
                option_id = option.get('option_id')
                option_name = option.get('option_name')

                # Check if this data exists in the database
                if page_id not in db_data:
                    missing_data.append(f"Page '{page_name}' (ID: {page_id}) is missing.")
                elif question_id not in db_data[page_id]:
                    missing_data.append(f"Question '{question_description}' (ID: {question_id}) is missing in page '{page_name}' (ID: {page_id}).")
                elif option_id not in db_data[page_id][question_id]:
                    missing_data.append(f"Option '{option_name}' (ID: {option_id}) for question '{question_description}' (ID: {question_id}) is missing in page '{page_name}' (ID: {page_id}).")

    # Output missing data
    if missing_data:
        print("Missing data:")
        for item in missing_data:
            print(item)
    else:
        print("All data is present.")

# Close the database connection
connection.close()


Список доступных файлов:
1. out_14_12_2024_АЭП.json
2. out_14_12_2024_АЭС.json
3. out_14_12_2024_БИТ.json
4. out_14_12_2024_ВМ.json
5. out_14_12_2024_ВМСС.json
6. out_14_12_2024_ВТ.json
7. out_14_12_2024_ГВИЭ.json
8. out_14_12_2024_ГГМ.json
9. out_14_12_2024_Дизайн.json
10. out_14_12_2024_ДИТ.json
11. out_14_12_2024_И и К.json
12. out_14_12_2024_Ин.яз.json
13. out_14_12_2024_ИТНО.json
14. out_14_12_2024_ИТФ.json
15. out_14_12_2024_ИЭиОТ.json
16. out_14_12_2024_МиПЭУ.json
17. out_14_12_2024_МКМ.json
18. out_14_12_2024_МЭП.json
19. out_14_12_2024_НТ.json
20. out_14_12_2024_ОРТ.json
21. out_14_12_2024_ОФиЯС.json
22. out_14_12_2024_ПГТ.json
23. out_14_12_2024_ПМИИ.json
24. out_14_12_2024_Пром.эл..json
25. out_14_12_2024_ПТС.json
26. out_14_12_2024_РЗиАЭ.json
27. out_14_12_2024_РМДиПМ.json
28. out_14_12_2024_РС и Л.json
29. out_14_12_2024_РТП и АС.json
30. out_14_12_2024_РТС.json
31. out_14_12_2024_Светотех..json
32. out_14_12_2024_ТМ.json
33. out_14_12_2024_ТМПУ.json
34. out_14_12_2024_ТОТ

In [14]:
import pyodbc
import json
import os

server = 'KPI-MONITOR'
database = 'MEI2'
username = 'efqm'
password = 'mpeiR@dar'

# Папка, содержащая файлы JSON
json_folder = 'JSON'

# Подключение к базе данных
connection_string = f'DRIVER=SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'
connection = pyodbc.connect(connection_string)

# Получаем список всех файлов в папке JSON
json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

# Запрашиваем у пользователя название файла для subdivision_name
print("Список доступных файлов:")
for idx, file in enumerate(json_files, 1):
    print(f"{idx}. {file}")

file_index = int(input("Введите номер файла, который хотите использовать: ")) - 1
filename = os.path.join(json_folder, json_files[file_index])

# Загружаем данные JSON из выбранного файла
with open(filename, 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

# Запрашиваем название подразделения для subdivision_name
subdivision_name = input("Введите название подразделения: ")

# Теперь у вас есть переменные filename (путь к файлу) и subdivision_name
print(f"Выбран файл: {filename}")
print(f"Подразделение: {subdivision_name}")

# Извлекаем все question_id из JSON данных
question_ids = set()

# Проходим по страницам и вопросам в JSON и извлекаем question_id
for page in json_data.get(subdivision_name, {}).get('pages', []):
    for question in page.get('questions', []):
        question_id = question.get('question_id')
        if question_id:
            question_ids.add(question_id)

# Выводим все уникальные question_id
if question_ids:
    print("Все уникальные question_id, найденные в JSON файле:")
    for question_id in sorted(question_ids):
        print(question_id)
else:
    print("В JSON файле не найдено question_id.")

Список доступных файлов:
1. out_14_12_2024_АЭП.json
2. out_14_12_2024_АЭС.json
3. out_14_12_2024_БИТ.json
4. out_14_12_2024_ВМ.json
5. out_14_12_2024_ВМСС.json
6. out_14_12_2024_ВТ.json
7. out_14_12_2024_ГВИЭ.json
8. out_14_12_2024_ГГМ.json
9. out_14_12_2024_Дизайн.json
10. out_14_12_2024_ДИТ.json
11. out_14_12_2024_И и К.json
12. out_14_12_2024_Ин.яз.json
13. out_14_12_2024_ИТНО.json
14. out_14_12_2024_ИТФ.json
15. out_14_12_2024_ИЭиОТ.json
16. out_14_12_2024_МиПЭУ.json
17. out_14_12_2024_МКМ.json
18. out_14_12_2024_МЭП.json
19. out_14_12_2024_НТ.json
20. out_14_12_2024_ОРТ.json
21. out_14_12_2024_ОФиЯС.json
22. out_14_12_2024_ПГТ.json
23. out_14_12_2024_ПМИИ.json
24. out_14_12_2024_Пром.эл..json
25. out_14_12_2024_ПТС.json
26. out_14_12_2024_РЗиАЭ.json
27. out_14_12_2024_РМДиПМ.json
28. out_14_12_2024_РС и Л.json
29. out_14_12_2024_РТП и АС.json
30. out_14_12_2024_РТС.json
31. out_14_12_2024_Светотех..json
32. out_14_12_2024_ТМ.json
33. out_14_12_2024_ТМПУ.json
34. out_14_12_2024_ТОТ

In [15]:
import pyodbc
import json
import os
import re

server = 'KPI-MONITOR'
database = 'MEI2'
username = 'efqm'
password = 'mpeiR@dar'

# Папка, содержащая файлы JSON
json_folder = 'JSON'

# Подключение к базе данных
connection_string = f'DRIVER=SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'
connection = pyodbc.connect(connection_string)

# Получаем список всех файлов в папке JSON
json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

# Запрашиваем у пользователя выбор файла
print("Список доступных файлов:")
for idx, file in enumerate(json_files, 1):
    print(f"{idx}. {file}")

file_index = int(input("Введите номер файла, который хотите использовать: ")) - 1
filename = os.path.join(json_folder, json_files[file_index])

# Извлекаем название подразделения (subdivision_name) из имени файла
# Пример: 'out_14_12_2024_АЭП.json' -> 'АЭП'
subdivision_name = re.search(r'_(\D+)\.json$', filename)
if subdivision_name:
    subdivision_name = subdivision_name.group(1)
    print(f"Подразделение: {subdivision_name}")
else:
    print("Не удалось извлечь название подразделения из имени файла.")
    subdivision_name = input("Введите название подразделения вручную: ")

# Загружаем данные JSON из выбранного файла
with open(filename, 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

# Теперь у вас есть переменные filename (путь к файлу) и subdivision_name
print(f"Выбран файл: {filename}")

# Извлекаем все question_id из JSON данных
question_ids = set()

# Проходим по страницам и вопросам в JSON и извлекаем question_id
for page in json_data.get(subdivision_name, {}).get('pages', []):
    for question in page.get('questions', []):
        question_id = question.get('question_id')
        if question_id:
            question_ids.add(question_id)

# Выводим все уникальные question_id
if question_ids:
    print("Все уникальные question_id, найденные в JSON файле:")
    for question_id in sorted(question_ids):
        print(question_id)
else:
    print("В JSON файле не найдено question_id.")


Список доступных файлов:
1. out_14_12_2024_АЭП.json
2. out_14_12_2024_АЭС.json
3. out_14_12_2024_БИТ.json
4. out_14_12_2024_ВМ.json
5. out_14_12_2024_ВМСС.json
6. out_14_12_2024_ВТ.json
7. out_14_12_2024_ГВИЭ.json
8. out_14_12_2024_ГГМ.json
9. out_14_12_2024_Дизайн.json
10. out_14_12_2024_ДИТ.json
11. out_14_12_2024_И и К.json
12. out_14_12_2024_Ин.яз.json
13. out_14_12_2024_ИТНО.json
14. out_14_12_2024_ИТФ.json
15. out_14_12_2024_ИЭиОТ.json
16. out_14_12_2024_МиПЭУ.json
17. out_14_12_2024_МКМ.json
18. out_14_12_2024_МЭП.json
19. out_14_12_2024_НТ.json
20. out_14_12_2024_ОРТ.json
21. out_14_12_2024_ОФиЯС.json
22. out_14_12_2024_ПГТ.json
23. out_14_12_2024_ПМИИ.json
24. out_14_12_2024_Пром.эл..json
25. out_14_12_2024_ПТС.json
26. out_14_12_2024_РЗиАЭ.json
27. out_14_12_2024_РМДиПМ.json
28. out_14_12_2024_РС и Л.json
29. out_14_12_2024_РТП и АС.json
30. out_14_12_2024_РТС.json
31. out_14_12_2024_Светотех..json
32. out_14_12_2024_ТМ.json
33. out_14_12_2024_ТМПУ.json
34. out_14_12_2024_ТОТ

In [9]:
pwd

'C:\\Efqm\\NewModel'

All data is present.


In [20]:
import pyodbc
import json
import os
import re

server = 'KPI-MONITOR'
database = 'MEI2'
username = 'efqm'
password = 'mpeiR@dar'

# Папка, содержащая файлы JSON
json_folder = 'JSON'

# Подключение к базе данных
connection_string = f'DRIVER=SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'
connection = pyodbc.connect(connection_string)

# Получаем список всех файлов в папке JSON
json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

# Запрашиваем у пользователя выбор файла
print("Список доступных файлов:")
for idx, file in enumerate(json_files, 1):
    print(f"{idx}. {file}")

file_index = int(input("Введите номер файла, который хотите использовать: ")) - 1
filename = os.path.join(json_folder, json_files[file_index])

# Извлекаем название подразделения (subdivision_name) из имени файла
# Пример: 'out_14_12_2024_АЭП.json' -> 'АЭП'
subdivision_name = re.search(r'_(\D+)\.json$', filename)
if subdivision_name:
    subdivision_name = subdivision_name.group(1)
    print(f"Подразделение: {subdivision_name}")
else:
    print("Не удалось извлечь название подразделения из имени файла.")
    subdivision_name = input("Введите название подразделения вручную: ")

# Загружаем данные JSON из выбранного файла
with open(filename, 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

# Теперь у вас есть переменные filename (путь к файлу) и subdivision_name
print(f"Выбран файл: {filename}")

# Извлекаем все question_id из JSON данных
question_ids = set()

# Проходим по страницам и вопросам в JSON и извлекаем question_id
for page in json_data.get(subdivision_name, {}).get('pages', []):
    for question in page.get('questions', []):
        question_id = question.get('question_id')
        if question_id:
            question_ids.add(question_id)

# Выводим все уникальные question_id
if question_ids:
    print("Все уникальные question_id, найденные в JSON файле:")
    for question_id in sorted(question_ids):
        print(question_id)
else:
    print("В JSON файле не найдено question_id.")

# Пример SQL-запроса, если нужно продолжить
cursor = connection.cursor()
sub_query = f"""
    SELECT page, page_id, question_id, question_description, option_id, option_name, answer
    FROM [MEI2].[dbo].[EFQM_SurveyAnswers]
    WHERE [SubdivisionShortName] = '{subdivision_name}'
"""
results = cursor.execute(sub_query).fetchall()

# Подготовка данных из базы данных для проверки
db_data = {}
for row in results:
    page = row.page
    page_id = row.page_id
    question_id = row.question_id
    option_id = row.option_id
    option_name = row.option_name

    if page_id not in db_data:
        db_data[page_id] = {}
    if question_id not in db_data[page_id]:
        db_data[page_id][question_id] = {}
    db_data[page_id][question_id][option_id] = option_name

# Проверка на наличие всех данных в базе данных
missing_data = []

# Проходим по JSON и проверяем, есть ли все данные в базе
for page in json_data.get(subdivision_name, {}).get('pages', []):
    page_id = page.get('page_id')
    page_name = page.get('page')

    # Проверяем вопросы на каждой странице
    for question in page.get('questions', []):
        question_id = question.get('question_id')
        question_description = question.get('question_description')

        # Проверяем варианты ответов для каждого вопроса
        for option in question.get('options', []):
            option_id = option.get('option_id')
            option_name = option.get('option_name')

            # Проверяем, есть ли это в базе данных
            if page_id not in db_data:
                missing_data.append(f"Page '{page_name}' (ID: {page_id}) is missing.")
            elif question_id not in db_data[page_id]:
                missing_data.append(f"Question '{question_description}' (ID: {question_id}) is missing in page '{page_name}' (ID: {page_id}).")
            elif option_id not in db_data[page_id][question_id]:
                missing_data.append(f"Option '{option_name}' (ID: {option_id}) for question '{question_description}' (ID: {question_id}) is missing in page '{page_name}' (ID: {page_id}).")

# Вывод недостающих данных
if missing_data:
    print("Missing data:")
    for item in missing_data:
        print(item)
else:
    print("All data is present.")

# Закрываем соединение с базой данных
connection.close()


Список доступных файлов:
1. out_14_12_2024_АЭП.json
2. out_14_12_2024_АЭС.json
3. out_14_12_2024_БИТ.json
4. out_14_12_2024_ВМ.json
5. out_14_12_2024_ВМСС.json
6. out_14_12_2024_ВТ.json
7. out_14_12_2024_ГВИЭ.json
8. out_14_12_2024_ГГМ.json
9. out_14_12_2024_Дизайн.json
10. out_14_12_2024_ДИТ.json
11. out_14_12_2024_И и К.json
12. out_14_12_2024_Ин.яз.json
13. out_14_12_2024_ИТНО.json
14. out_14_12_2024_ИТФ.json
15. out_14_12_2024_ИЭиОТ.json
16. out_14_12_2024_МиПЭУ.json
17. out_14_12_2024_МКМ.json
18. out_14_12_2024_МЭП.json
19. out_14_12_2024_НТ.json
20. out_14_12_2024_ОРТ.json
21. out_14_12_2024_ОФиЯС.json
22. out_14_12_2024_ПГТ.json
23. out_14_12_2024_ПМИИ.json
24. out_14_12_2024_Пром.эл..json
25. out_14_12_2024_ПТС.json
26. out_14_12_2024_РЗиАЭ.json
27. out_14_12_2024_РМДиПМ.json
28. out_14_12_2024_РС и Л.json
29. out_14_12_2024_РТП и АС.json
30. out_14_12_2024_РТС.json
31. out_14_12_2024_Светотех..json
32. out_14_12_2024_ТМ.json
33. out_14_12_2024_ТМПУ.json
34. out_14_12_2024_ТОТ

In [24]:
import pyodbc
import json
import os
import re

server = 'KPI-MONITOR'
database = 'MEI2'
username = 'efqm'
password = 'mpeiR@dar'

# Папка, содержащая файлы JSON
json_folder = 'JSON'

# Подключение к базе данных
connection_string = f'DRIVER=SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'
connection = pyodbc.connect(connection_string)

# Получаем список всех файлов в папке JSON
json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

# Запрашиваем у пользователя выбор файла
print("Список доступных файлов:")
for idx, file in enumerate(json_files, 1):
    print(f"{idx}. {file}")

file_index = int(input("Введите номер файла, который хотите использовать: ")) - 1
filename = os.path.join(json_folder, json_files[file_index])

# Извлекаем название подразделения (subdivision_name) из имени файла
# Пример: 'out_14_12_2024_АЭП.json' -> 'АЭП'
subdivision_name = re.search(r'_(\D+)\.json$', filename)
if subdivision_name:
    subdivision_name = subdivision_name.group(1)
    print(f"Подразделение: {subdivision_name}")
else:
    print("Не удалось извлечь название подразделения из имени файла.")
    subdivision_name = input("Введите название подразделения вручную: ")

# Загружаем данные JSON из выбранного файла
with open(filename, 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

# Теперь у вас есть переменные filename (путь к файлу) и subdivision_name
print(f"Выбран файл: {filename}")

# Извлекаем все question_id из JSON данных
question_ids = set()

# Проходим по страницам и вопросам в JSON и извлекаем question_id
for page in json_data.get(subdivision_name, {}).get('pages', []):
    for question in page.get('questions', []):
        question_id = question.get('question_id')
        if question_id:
            question_ids.add(question_id)

# Выводим все уникальные question_id
if question_ids:
    print("Все уникальные question_id, найденные в JSON файле:")
    for question_id in sorted(question_ids):
        print(question_id)
else:
    print("В JSON файле не найдено question_id.")

# Пример SQL-запроса с агрегацией для уникальных option_id с максимальным значением по answer
sub_query = f"""
    WITH RankedAnswers AS (
        SELECT 
            page, 
            page_id, 
            question_id, 
            question_description, 
            option_id, 
            option_name, 
            answer,
            ROW_NUMBER() OVER (PARTITION BY question_id, option_id ORDER BY answer DESC) AS rn
        FROM [MEI2].[dbo].[EFQM_SurveyAnswers]
        WHERE [SubdivisionShortName] = '{subdivision_name}'
    )
    SELECT 
        page, 
        page_id, 
        question_id, 
        question_description, 
        option_id, 
        option_name, 
        answer
    FROM RankedAnswers
    WHERE rn = 1
    ORDER BY question_id, option_id;
"""

# Выполнение запроса
cursor = connection.cursor()
results = cursor.execute(sub_query).fetchall()

# Подготовка данных из базы данных для проверки
db_data = {}
for row in results:
    page = row.page
    page_id = row.page_id
    question_id = row.question_id
    option_id = row.option_id
    option_name = row.option_name

    if page_id not in db_data:
        db_data[page_id] = {}
    if question_id not in db_data[page_id]:
        db_data[page_id][question_id] = {}
    db_data[page_id][question_id][option_id] = option_name

# Проверка на наличие всех данных в базе данных
missing_data = []

# Проходим по JSON и проверяем, есть ли все данные в базе
for page in json_data.get(subdivision_name, {}).get('pages', []):
    page_id = page.get('page_id')
    page_name = page.get('page')

    # Проверяем вопросы на каждой странице
    for question in page.get('questions', []):
        question_id = question.get('question_id')
        question_description = question.get('question_description')

        # Проверяем варианты ответов для каждого вопроса
        for option in question.get('options', []):
            option_id = option.get('option_id')
            option_name = option.get('option_name')

            # Проверяем, есть ли это в базе данных
            if page_id not in db_data:
                missing_data.append(f"Page '{page_name}' (ID: {page_id}) is missing.")
            elif question_id not in db_data[page_id]:
                missing_data.append(f"Question '{question_description}' (ID: {question_id}) is missing in page '{page_name}' (ID: {page_id}).")
            elif option_id not in db_data[page_id][question_id]:
                missing_data.append(f"Option '{option_name}' (ID: {option_id}) for question '{question_description}' (ID: {question_id}) is missing in page '{page_name}' (ID: {page_id}).")

# Вывод недостающих данных
if missing_data:
    print("Missing data:")
    for item in missing_data:
        print(item)
else:
    print("All data is present.")

# Закрываем соединение с базой данных
connection.close()


Список доступных файлов:
1. out_14_12_2024_АЭП.json
2. out_14_12_2024_АЭС.json
3. out_14_12_2024_БИТ.json
4. out_14_12_2024_ВМ.json
5. out_14_12_2024_ВМСС.json
6. out_14_12_2024_ВТ.json
7. out_14_12_2024_ГВИЭ.json
8. out_14_12_2024_ГГМ.json
9. out_14_12_2024_Дизайн.json
10. out_14_12_2024_ДИТ.json
11. out_14_12_2024_И и К.json
12. out_14_12_2024_Ин.яз.json
13. out_14_12_2024_ИТНО.json
14. out_14_12_2024_ИТФ.json
15. out_14_12_2024_ИЭиОТ.json
16. out_14_12_2024_МиПЭУ.json
17. out_14_12_2024_МКМ.json
18. out_14_12_2024_МЭП.json
19. out_14_12_2024_НТ.json
20. out_14_12_2024_ОРТ.json
21. out_14_12_2024_ОФиЯС.json
22. out_14_12_2024_ПГТ.json
23. out_14_12_2024_ПМИИ.json
24. out_14_12_2024_Пром.эл..json
25. out_14_12_2024_ПТС.json
26. out_14_12_2024_РЗиАЭ.json
27. out_14_12_2024_РМДиПМ.json
28. out_14_12_2024_РС и Л.json
29. out_14_12_2024_РТП и АС.json
30. out_14_12_2024_РТС.json
31. out_14_12_2024_Светотех..json
32. out_14_12_2024_ТМ.json
33. out_14_12_2024_ТМПУ.json
34. out_14_12_2024_ТОТ

In [120]:
import pyodbc
import json
import os
import re

server = 'KPI-MONITOR'
database = 'MEI2'
username = 'efqm'
password = 'mpeiR@dar'

# Папка, содержащая файлы JSON
json_folder = 'JSON'

# Подключение к базе данных
connection_string = f'DRIVER=SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'
connection = pyodbc.connect(connection_string)

# Получаем список всех файлов в папке JSON
json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

# Запрашиваем у пользователя выбор файла
print("Список доступных файлов:")
for idx, file in enumerate(json_files, 1):
    print(f"{idx}. {file}")

file_index = int(input("Введите номер файла, который хотите использовать: ")) - 1
filename = os.path.join(json_folder, json_files[file_index])

# Извлекаем название подразделения (subdivision_name) из имени файла
subdivision_name = re.search(r'_(\D+)\.json$', filename)
if subdivision_name:
    subdivision_name = subdivision_name.group(1)
    print(f"Подразделение: {subdivision_name}")
else:
    print("Не удалось извлечь название подразделения из имени файла.")
    subdivision_name = input("Введите название подразделения вручную: ")

# Загружаем данные JSON из выбранного файла
with open(filename, 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

print(f"Выбран файл: {filename}")

# Извлекаем данные из базы данных, аналогичные структуре в JSON
cursor = connection.cursor()

# SQL-запрос для извлечения всех вопросов из базы данных
sub_query = f"""
    SELECT 
        page, 
        page_id, 
        question_id, 
        question_description
    FROM [MEI2].[dbo].[EFQM_SurveyAnswers]
    WHERE [SubdivisionShortName] = '{subdivision_name}'
    GROUP BY page, page_id, question_id, question_description
    ORDER BY question_id;
"""

results = cursor.execute(sub_query).fetchall()

# Подготовка данных из базы данных для проверки
db_question_ids = set()
for row in results:
    question_id = row.question_id
    db_question_ids.add(question_id)

# Извлекаем все question_id из JSON данных
json_question_ids = set()

for page in json_data.get(subdivision_name, {}).get('pages', []):
    for question in page.get('questions', []):
        question_id = question.get('question_id')
        if question_id:
            json_question_ids.add(question_id)

# Сравнение: выводим все question_id, которые есть в базе данных, но нет в JSON
missing_data = []

for question_id in db_question_ids:
    if question_id not in json_question_ids:
        missing_data.append(f"Question ID '{question_id}' is missing in the JSON.")

# Вывод недостающих данных
if missing_data:
    print("Missing question_ids in JSON:")
    for item in missing_data:
        print(item)
else:
    print("All question_ids from the database are present in the JSON.")

# Закрываем соединение с базой данных
connection.close()


Список доступных файлов:
1. out_14_12_2024_АСУТП.json
2. out_14_12_2024_АЭП.json
3. out_14_12_2024_АЭС.json
4. out_14_12_2024_БИТ.json
5. out_14_12_2024_ВМ.json
6. out_14_12_2024_ВМСС.json
7. out_14_12_2024_ВТ.json
8. out_14_12_2024_ГВИЭ.json
9. out_14_12_2024_ГГМ.json
10. out_14_12_2024_Дизайн.json
11. out_14_12_2024_ДИТ.json
12. out_14_12_2024_И и К.json
13. out_14_12_2024_Ин.яз.json
14. out_14_12_2024_ИТНО.json
15. out_14_12_2024_ИТФ.json
16. out_14_12_2024_ИЭиОТ.json
17. out_14_12_2024_МиПЭУ.json
18. out_14_12_2024_МКМ.json
19. out_14_12_2024_МЭП.json
20. out_14_12_2024_НТ.json
21. out_14_12_2024_ОРТ.json
22. out_14_12_2024_ОФиЯС.json
23. out_14_12_2024_ПГТ.json
24. out_14_12_2024_ПМИИ.json
25. out_14_12_2024_Пром.эл..json
26. out_14_12_2024_ПТС.json
27. out_14_12_2024_РЗиАЭ.json
28. out_14_12_2024_РМДиПМ.json
29. out_14_12_2024_РС и Л.json
30. out_14_12_2024_РТП и АС.json
31. out_14_12_2024_РТС.json
32. out_14_12_2024_Светотех..json
33. out_14_12_2024_ТМ.json
34. out_14_12_2024_ТМ